<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [1]:
import numpy as np
import pandas as pd
import gensim
import os
import re
import zipfile

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

C:\Users\navo1\Anaconda3\envs\U4-S1-NLP\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Load the reviews

In [7]:
path = "data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv"

reviews = pd.read_csv(path)
reviews.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [8]:
STOPWORDS = set(STOPWORDS).union(set(['amazon']))

def tokenize(text):
    return [token for token in simple_preprocess(text, deacc=True, min_len=4, max_len=20) if token not in STOPWORDS]

In [9]:
def wrangle(df):
    new_df = df.copy()
    new_df['brand'] = new_df['brand'].apply(lambda x: x.lower())
    new_df['tokens'] = new_df['reviews.text'].apply(tokenize)
    new_df = new_df[['brand', 'reviews.text', 'tokens']]
    return new_df

df = wrangle(reviews)
df.head()

,brand,reviews.text,tokens
0,amazonbasics,I order 3 of them and one of the item is bad q...,"[order, item, quality, missing, backup, spring..."
1,amazonbasics,Bulk is always the less expensive way to go fo...,"[bulk, expensive, products, like]"
2,amazonbasics,Well they are not Duracell but for the price i...,"[duracell, price, happy]"
3,amazonbasics,Seem to work as well as name brand batteries a...,"[work, brand, batteries, better, price]"
4,amazonbasics,These batteries are very long lasting the pric...,"[batteries, long, lasting, price, great]"


# LDA Topic Modeling

In [10]:
def get_reviews(df, token_col):
    
    assert token_col in df.columns, f"{token_col} does not exist!"
    
    for index, row in df.iterrows():
        review_token = row[token_col]
        yield review_token

In [11]:
id2word = corpora.Dictionary(get_reviews(df, 'tokens'))
id2word.token2id['batteries']

16

In [12]:
sample_tokens = tokenize("I order 3 of them and one of the item is bad quality. Is missing backup spring so I have to put a pcs of aluminum to make the battery work.")
sample_tokens

['order',
 'item',
 'quality',
 'missing',
 'backup',
 'spring',
 'aluminum',
 'battery',
 'work']

In [13]:
id2word.doc2bow(sample_tokens)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]

In [14]:
# a bag of words(bow) representation of our corpus
# Note: we haven't actually read any text into memory here
corpus = [id2word.doc2bow(tokens) for tokens in get_reviews(df, 'tokens')]
corpus[0][:10]

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]

In [15]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=4
                  )

In [16]:
def print_lda_topics(lda):
    words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]
    topics = [' '.join(t[0:5]) for t in words]
    for id, t in enumerate(topics): 
        print(f"------ Topic {id} ------")
        print(t, end="\n\n")
print_lda_topics(lda)

------ Topic 0 ------
purchase batteries battery happy pleased

------ Topic 1 ------
tablet great size good screen

------ Topic 2 ------
battery like batteries duracell long

------ Topic 3 ------
love loves year bought games

------ Topic 4 ------
kindle great excellent black sound

------ Topic 5 ------
games easy play like loves

------ Topic 6 ------
works easy great read user

------ Topic 7 ------
loves bought daughter tablet wife

------ Topic 8 ------
great good price product batteries

------ Topic 9 ------
good charge kindle screen like

------ Topic 10 ------
tablet love great easy perfect

------ Topic 11 ------
gift bought recommend best loved

------ Topic 12 ------
kids tablet apps great play

------ Topic 13 ------
books read kindle reading tablet

------ Topic 14 ------
batteries long work brand battery



## Topic Distance Visualization

In [18]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [19]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

C:\Users\navo1\Anaconda3\envs\U4-S1-NLP\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14     0.255783 -0.084266       1        1  15.470934
8      0.210668  0.055728       2        1   9.472336
12    -0.123390 -0.018295       3        1   9.315135
9      0.048113  0.083451       4        1   8.072384
1     -0.037988  0.165154       5        1   7.786201
3     -0.198235 -0.196248       6        1   6.988729
4      0.021866  0.149607       7        1   5.603185
6     -0.109589  0.192418       8        1   5.351733
13    -0.129747  0.079079       9        1   4.951789
2      0.229116 -0.051897      10        1   4.939498
7     -0.066137 -0.113778      11        1   4.784161
10    -0.123196 -0.017440      12        1   4.556542
11    -0.059072 -0.104614      13        1   4.375944
5     -0.131992 -0.071492      14        1   4.221289
0      0.213799 -0.067408      15        1   4.110137, topic_info=     Category         Freq         Term        Total  loglift  logprob
3889  Default  6855.000000       tablet  6855.000000  30.0000  30.0000
16    Default  6965.000000    batteries  6965.000000  29.0000  29.0000
15    Default  4758.000000        price  4758.000000  28.0000  28.0000
30    Default  5440.000000         good  5440.000000  27.0000  27.0000
19    Default  8829.000000        great  8829.000000  26.0000  26.0000
2424  Default  2204.000000        loves  2204.000000  25.0000  25.0000
2923  Default  3089.000000       kindle  3089.000000  24.0000  24.0000
2936  Default  1534.000000        books  1534.000000  23.0000  23.0000
711   Default  1747.000000        games  1747.000000  22.0000  22.0000
25    Default  3109.000000       bought  3109.000000  21.0000  21.0000
179   Default  2422.000000         kids  2422.000000  20.0000  20.0000
831   Default  2976.000000         easy  2976.000000  19.0000  19.0000
81    Default  3366.000000         love  3366.000000  18.0000  18.0000
981   Default  1048.000000         gift  1048.000000  17.0000  17.0000
863   Default  1232.000000         read  1232.000000  16.0000  16.0000
1488  Default   972.000000     daughter   972.000000  15.0000  15.0000
92    Default  1769.000000         year  1769.000000  14.0000  14.0000
155   Default  2097.000000      product  2097.000000  13.0000  13.0000
421   Default  1148.000000         play  1148.000000  12.0000  12.0000
86    Default  1417.000000        works  1417.000000  11.0000  11.0000
399   Default   978.000000        value   978.000000  10.0000  10.0000
1899  Default  1797.000000         apps  1797.000000   9.0000   9.0000
2     Default  2041.000000      battery  2041.000000   8.0000   8.0000
1059  Default  1211.000000      reading  1211.000000   7.0000   7.0000
27    Default   866.000000    christmas   866.000000   6.0000   6.0000
117   Default   927.000000         size   927.000000   5.0000   5.0000
18    Default  1202.000000        brand  1202.000000   4.0000   4.0000
21    Default  2332.000000         long  2332.000000   3.0000   3.0000
8     Default  2208.000000         work  2208.000000   2.0000   2.0000
35    Default   981.000000     purchase   981.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
1212  Topic15    62.737686     customer   116.668182   2.5713  -5.2186
768   Topic15    33.942448      shipped    54.014111   2.7271  -5.8328
1723  Topic15    36.263458        tough    60.129169   2.6860  -5.7667
663   Topic15    72.008072    impressed   152.337158   2.4424  -5.0807
467   Topic15   123.337914      quickly   326.705719   2.2176  -4.5426
36    Topic15    82.270706      ordered   192.386307   2.3422  -4.9475
5     Topic15   100.280647        order   300.982147   2.0926  -4.7495
98    Topic15    68.991600      exactly   177.470184   2.2469  -5.1235
235   Topic15   114.687515  performance   384.605103   1.9817  -4.6153
14    Topic15   181.979202        happy   824.762695   1.6805  -4.1536
384   Topic15    84.362076        super   253.577438   2.

In [20]:
cm = CoherenceModel(model=lda, corpus=corpus, coherence='u_mass')
print(f'Coherence score: {cm.get_coherence()}')

NameError: name 'CoherenceModel' is not defined

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling